In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
import nltk

In [138]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# DATA EXTRACTION

In [2]:
files = glob.glob('/home/varun/Back up/digitalV/Capstone/train/neg/*.txt')
rawData=[]
for file in files:
    Data = open(file).read()
    rawData.append(Data)   

In [3]:
negDataTrain = pd.DataFrame(rawData)
negDataTrain.columns = ['review']
negDataTrain['label']= 0
negDataTrain.head()

,review,label
0,Tourists head to Ireland for a school trip to ...,0
1,Here's how you do it: Believe in God and repen...,0
2,A major disappointment. This was one of the be...,0
3,Worst show I've ever seen. The story is about ...,0
4,"So, Prom Night was supposed to be a horror and...",0


In [4]:
files1 = glob.glob('/home/varun/Back up/digitalV/Capstone/train/pos/*.txt')
rawData=[]
for file in files1:
    Data = open(file).read()
    rawData.append(Data)

In [5]:
posDataTrain = pd.DataFrame(rawData)
posDataTrain.columns = ['review']
posDataTrain['label']= 1
posDataTrain.head()

,review,label
0,"I saw this movie on TV when it came out, and n...",1
1,All dogs go to Heaven is one of the best movie...,1
2,I have decided to not believe what famous movi...,1
3,This short is a puzzlement. Words fail me here...,1
4,Felix Unger (Jack Lemmon) has just been dumped...,1


In [6]:
from sklearn.utils import shuffle

train_df = pd.concat([posDataTrain,negDataTrain])
train_df = shuffle(train_df).reset_index(drop=True)
train_df.head()

,review,label
0,I run a group to stop comedian exploitation an...,0
1,I dont know why people think this is such a ba...,1
2,Especially for a time when not much science fi...,1
3,I'm dumbfounded. Yes that's right. I'm really ...,0
4,"This really should have been a one star, but t...",0


In [7]:
files = glob.glob('/home/varun/Back up/digitalV/Capstone/test/neg/*.txt')
rawData=[]
for file in files:
    Data = open(file).read()
    rawData.append(Data)

In [8]:
negDataTest = pd.DataFrame(rawData)
negDataTest.columns = ['review']
negDataTest['label']= 0
negDataTest.head()

,review,label
0,I just finished watching Dog Watch. I thought ...,0
1,Seems to me that Joe Estevez spends most of hi...,0
2,"Very simply, they are all the syndicated episo...",0
3,Very strange screenplay by Cameron Crowe (foll...,0
4,"Miranda Cosgrove is known for her debut in ""Sc...",0


In [9]:
files = glob.glob('/home/varun/Back up/digitalV/Capstone/test/pos/*.txt')
rawData=[]
for file in files:
    Data = open(file).read()
    rawData.append(Data)

In [10]:
posDataTest = pd.DataFrame(rawData)
posDataTest.columns = ['review']
posDataTest['label']= 1
posDataTest.head()

,review,label
0,I don't normally go out of my way to watch rom...,1
1,"In the year 2000 (keep in mind, this is two ye...",1
2,It was riveting. I just could not look away. A...,1
3,The Beguiled was one of the few early Eastwood...,1
4,For a TV movie this was definately worth seein...,1


In [11]:
from sklearn.utils import shuffle

test_df = pd.concat([posDataTest,negDataTest])
test_df = shuffle(test_df).reset_index(drop=True)
test_df.head()

,review,label
0,The theme song often goes through my head afte...,1
1,Tim Robbins makes a wonderful film. His wife (...,1
2,This movies had to be the funniest movie i hav...,1
3,"Without saying how it ended, it is sufficient ...",0
4,Yes. Bam cried a couple times and so did Englu...,1


# CLEANING

In [12]:
import string
import re
from nltk.stem import WordNetLemmatizer
clean = re.compile('<.*?>')
ps = nltk.PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')
lem = WordNetLemmatizer()

def cleanText(text):
    html = re.sub(clean,'',text)
    p_text = ''.join([i.lower() for i in html if i not in string.punctuation])
    token = re.split('\W+',p_text)
    stop = [ps.stem(i) for i in token if i not in stopwords]
    #stop = [lem.lemmatize(i) for i in token if i not in stopwords]
    return stop
    

In [13]:
train_df['refined review'] = train_df['review'].apply(lambda x : cleanText(x) )
train_df.head()

,review,label,refined review
0,I run a group to stop comedian exploitation an...,0,"[run, group, stop, comedian, exploit, spent, p..."
1,I dont know why people think this is such a ba...,1,"[dont, know, peopl, think, bad, movi, got, pre..."
2,Especially for a time when not much science fi...,1,"[especi, time, much, scienc, fiction, film, 19..."
3,I'm dumbfounded. Yes that's right. I'm really ...,0,"[im, dumbfound, ye, that, right, im, realli, c..."
4,"This really should have been a one star, but t...",0,"[realli, one, star, mani, cliché, predict, twi..."


# FEATURE EXTRACTION

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=cleanText)
x_train_counts = count_vect.fit_transform(train_df['review'])

In [16]:
x_train_counts

<25000x112775 sparse matrix of type '<class 'numpy.int64'>'
	with 2423394 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
rfe = RFE(logReg,30)

In [ ]:
rfe = rfe.fit(x_train_counts[0:25],train_df['label'][0:25])

In [368]:
support = rfe.support_
ranking = rfe.ranking_

In [369]:
value = []
for i in range(len(ranking)):
    if ranking[i]==1:
        
        value.append(count_vect.get_feature_names()[i])

In [370]:
print(value[0:30])

['bad', 'bet', 'bulli', 'charact', 'even', 'far', 'friend', 'great', 'hang', 'im', 'joe', 'know', 'like', 'lta', 'luci', 'man', 'mean', 'movi', 'red', 'role', 'say', 'scene', 'shot', 'sit', 'someon', 'star', 'stori', 'two', 'want', 'year']


# EXPLORATION

In [18]:
def unify(column):
    unifiedString = column.str.cat(sep=' ') 
    unifiedString = cleanText(unifiedString)
    return unifiedString

In [19]:
word_dist_ntr = nltk.FreqDist(unify(negDataTrain['review']))
word_dist_neg_train = pd.DataFrame(word_dist_ntr.most_common(),columns=['Word','Frequency'])
print('Most common words in negative training data sets are :')
word_dist_neg_train.iloc[0:9,:]

Most common words in negative training data sets are :


,Word,Frequency
0,movi,27791
1,film,21895
2,one,12805
3,like,11961
4,make,7832
5,get,7600
6,even,7600
7,good,7229
8,time,7194


In [20]:
word_dist_ptr = nltk.FreqDist(unify(posDataTrain['review']))
word_dist_pos_train = pd.DataFrame(word_dist_ptr.most_common(),columns=['Word','Frequency'])
print('Most common words in positive training data sets are :')
word_dist_pos_train.iloc[0:9,:]

Most common words in positive training data sets are :


,Word,Frequency
0,film,24400
1,movi,21787
2,one,13508
3,like,10094
4,time,7883
5,good,7478
6,see,7250
7,stori,7188
8,charact,6867


In [21]:
word_dist_nt = nltk.FreqDist(unify(negDataTest['review']))
word_dist_neg_test = pd.DataFrame(word_dist_nt.most_common(),columns=['Word','Frequency'])
print('Most common words in negative testing data sets are :')
word_dist_neg_test.iloc[0:9,:]

Most common words in negative testing data sets are :


,Word,Frequency
0,movi,28040
1,film,21160
2,one,12893
3,like,11960
4,make,7566
5,get,7532
6,even,7487
7,watch,7391
8,time,7139


In [22]:
word_dist_pt = nltk.FreqDist(unify(posDataTest['review']))
word_dist_pos_test = pd.DataFrame(word_dist_pt.most_common(),columns=['Word','Frequency'])
print('Most common words in positive testing data sets are :')
word_dist_pos_test.iloc[0:9,:]

Most common words in positive testing data sets are :


,Word,Frequency
0,film,24596
1,movi,21297
2,one,13435
3,like,9787
4,time,7577
5,see,7433
6,good,7080
7,charact,6757
8,make,6500


In [23]:
word_dist_tr = nltk.FreqDist(unify(train_df['review']))
word_dist_train = pd.DataFrame(word_dist_tr.most_common(),columns=['Word','Frequency'])
print('Words with highest frequency in training data sets are : ')
word_dist_train.head(10)

Words with highest frequency in training data sets are : 


,Word,Frequency
0,movi,49570
1,film,46295
2,one,26311
3,like,22050
4,time,15078
5,good,14707
6,make,14492
7,get,14021
8,charact,13797
9,see,13759


In [24]:
print('Words with lowest frequency in training data sets are : ')
word_dist_train.tail(10)

Words with lowest frequency in training data sets are : 


,Word,Frequency
112752,sublimelyth,1
112753,punchingbag,1
112754,leastinspir,1
112755,hoariest,1
112756,blowth,1
112757,boxerther,1
112758,6200,1
112759,duello,1
112760,vc,1
112761,artemi,1


In [25]:
word_dist_t = nltk.FreqDist(unify(test_df['review']))
word_dist_test = pd.DataFrame(word_dist_t.most_common(),columns=['Word','Frequency'])
print('Words with highest frequency in testing data sets are : ')
word_dist_test.head(10)

Words with highest frequency in testing data sets are : 


,Word,Frequency
0,movi,49342
1,film,45761
2,one,26336
3,like,21753
4,time,14718
5,good,14183
6,make,14070
7,see,13800
8,charact,13768
9,get,13691


In [26]:
print('Words with lowest frequency in testing data sets are : ')
word_dist_test.tail(10)

Words with lowest frequency in testing data sets are : 


,Word,Frequency
110063,postbik,1
110064,itadam,1
110065,wastedbut,1
110066,nonbik,1
110067,nonsmok,1
110068,nonbooz,1
110069,flickshemp,1
110070,500000which,1
110071,thewinn,1
110072,thoughjust,1


# DATA EXTRACTION AND CLEANING FOR UNLABELED DATA

In [27]:
files_unsup = glob.glob('/home/varun/Back up/digitalV/Capstone/train/unsup/*.txt')
rawData = []
for file in files_unsup:
    data = open(file).read()
    rawData.append(data)  

In [28]:
unsup_df = pd.DataFrame(rawData)
unsup_df.columns = ['review']
unsup_df['refined review'] = unsup_df['review'].apply(lambda x : cleanText(x) )
unsup_df.head()

,review,refined review
0,I saw this on television recently. I remember ...,"[saw, televis, recent, rememb, saw, first, ear..."
1,"This English re-titled ""A Reason to Live, A Re...","[english, retitl, reason, live, reason, die, t..."
2,There is NO Esperanto in this movie<br /><br /...,"[esperanto, moviei, watch, movi, specif, imdb,..."
3,"The sweetest movie I've seen all year, full of...","[sweetest, movi, ive, seen, year, full, funni,..."
4,"Poorly written, poorly plotted, poorly acted, ...","[poorli, written, poorli, plot, poorli, act, p..."


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

tfIdf = TfidfVectorizer(analyzer=cleanText)
x_tfIdf = tfIdf.fit_transform(unsup_df['review'])

kmeans = KMeans(n_clusters=2, n_init=20,max_iter=1000)
kmeans.fit(x_tfIdf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=2, n_init=20, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [30]:
kmeans.cluster_centers_

array([[2.37251503e-03, 3.08900326e-04, 4.70828393e-05, ...,
        5.55131215e-06, 2.77565608e-06, 5.23842923e-06],
       [2.64479746e-03, 1.03556836e-03, 7.22673595e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [31]:
prediction = kmeans.predict(x_tfIdf)
prediction[0:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1], dtype=int32)

In [32]:
unsup_df['label'] = prediction
unsup_df.head(10)

,review,refined review,label
0,I saw this on television recently. I remember ...,"[saw, televis, recent, rememb, saw, first, ear...",0
1,"This English re-titled ""A Reason to Live, A Re...","[english, retitl, reason, live, reason, die, t...",0
2,There is NO Esperanto in this movie<br /><br /...,"[esperanto, moviei, watch, movi, specif, imdb,...",0
3,"The sweetest movie I've seen all year, full of...","[sweetest, movi, ive, seen, year, full, funni,...",0
4,"Poorly written, poorly plotted, poorly acted, ...","[poorli, written, poorli, plot, poorli, act, p...",0
5,"When 24 first premiered, I was quite skeptical...","[24, first, premier, quit, skeptic, would, las...",0
6,"By chance, this movie caught my attention late...","[chanc, movi, caught, attent, latenight, lucki...",0
7,"This is a low budget, indie horror film that t...","[low, budget, indi, horror, film, take, downwa...",0
8,This documentary is fundamentally misleading i...,"[documentari, fundament, mislead, character, s...",0
9,"Hollywood is known for its summertime action ""...","[hollywood, known, summertim, action, flick, r...",1


In [33]:
posList = train_df[train_df['label']== 1]['review'].tolist()
pos = " ".join(posList)
pos = cleanText(pos)

posStr = ""
posStr = ''.join(str(pos))
posStr[0:36]    

"['dont', 'know', 'peopl', 'think', '"